In [2]:
from lib.textbook_manager import TextbookManager

# Initialize manager
textbook_manager = TextbookManager()

# Print all available textbooks


# Get a specific textbook and page
textbook_name = "Applied Combinatorics (6th Edition) by Alan Tucker 2012 PDF"  # without .txt extension
passage = textbook_manager.get_page(textbook_name, page_number=211)
# Get and combine multiple consecutive pages
start_page = 211
num_pages = 3
combined_passage = None

for page_num in range(start_page, start_page + num_pages):
    current_passage = textbook_manager.get_page(textbook_name, page_number=page_num)
    if current_passage:
        if combined_passage is None:
            combined_passage = current_passage
        else:
            combined_passage.content += "\n\n" + current_passage.content
    else:
        print(f"Warning: Could not find page {page_num}")

print(f"Combined content from pages {start_page} to {start_page + num_pages - 1}:")
if passage:
    print(f"\nPage {passage.page_number} content:")
    print(passage.content)
else:
    print(f"Could not find textbook or page")


Combined content from pages 211 to 213:

Page 211 content:
Exponential Generating Functions 271
(c)Show that
(1+x)(1+x3)(1+x5)...
=1+∞/summationdisplay
k=1xk2
(1−x2)(1−x4)(1−x6)...(1−x2k)
22.Let/braceleftbign
k/bracerightbig
denote the number of partitions of ndistinct objects into knonempty
subsets. Show that/braceleftbign+1
k/bracerightbig
=k/braceleftbign
k/bracerightbig
+/braceleftbign
k−1/bracerightbig
.
23.Write a computer program to determine the number of all partitions of an inte-
gerr
(a)Intokparts
(b)Into any number of parts
(c)Into distinct parts
6.4 EXPONENTIAL GENERATING FUNCTIONS
In this section we discuss exponential generating functions. They are used to modeland solve problems involving arrangements with repetition. The generating functionsused in the previous sections to model selection with repetition problems are calledordinary generating functions. Exponential generating functions involve a more
complicated modeling process than do ordinary generating functions. C

In [7]:
from lib.question_generator import QuestionGenerator
from lib.question_generator.models import QuestionDifficulty

# Create question generator
generator = QuestionGenerator(model_name="gpt-4o-mini", verification_model="gpt-4o-mini")

# Generate questions with verification
print("Generating questions without verification...")
verified_questions = await generator.generate_questions(
    combined_passage.content,
    num_questions=10,
    difficulty=QuestionDifficulty.GRAD,  # or any other difficulty level
    verify=True,
  
)

print(f"\nGenerated {len(verified_questions)} verified questions:")
for i, qa in enumerate(verified_questions, 1):
    print(f"\n Source: {qa.source}")
    print(f"\nQuestion {i}:")
    print(qa.question)
    print("\nSolution:")
    print(qa.solution)
    
    print("-" * 80)
    



Generating questions without verification...
MODEL OUTPUT:  Here are the quiz questions based on the provided textbook snippet:

<question>
Consider the problem of finding the number of different words formed from an unlimited supply of letters a, b, and c, where the word must contain at least two a's and consists of four letters. How many different arrangements are possible?
</question>
<solution>
There are six possible selections of letters that meet the criteria: 
- {a, a, a, a}
- {a, a, a, b}
- {a, a, a, c}
- {a, a, b, b}
- {a, a, b, c}
- {a, a, c, c}

The number of arrangements for each selection:
- {a, a, a, a}: 1 arrangement
- {a, a, a, b}: 4!/(3!1!) = 4 arrangements
- {a, a, a, c}: 4!/(3!1!) = 4 arrangements
- {a, a, b, b}: 4!/(2!2!) = 6 arrangements
- {a, a, b, c}: 4!/(2!1!1!) = 12 arrangements
- {a, a, c, c}: 4!/(2!2!) = 6 arrangements

Total arrangements = 1 + 4 + 4 + 6 + 12 + 6 = 33 arrangements.

Thus, the answer is \(\boxed{33}\).
</solution>

<question>
What is the expon

In [1]:
import os
textbook_names = [f.replace('.txt', '') for f in os.listdir('textbooks/txt') if f.endswith('.txt')]
textbook_names

['pdfcoffee.com_the-art-of-problem-solving-intermediate-algebra-pdf-free',
 'dokumen.pub_introduction-to-counting-amp-probability-the-art-of-problem-solving-2nbsped-1934124109-9781934124109',
 'pdfcoffee.com_introduction-to-number-theory-mathew-crawford-pdf-free (1)',
 'mathematical-olympiad-treasures-2ed-springer-2011',
 '[Geoffrey_R._Grimmett,_David_R._Stirzaker]_Probabi(BookZZ.org)',
 'cameron_p.j._introduction_to_algebra',
 'pdfcoffee.com_introduction-to-number-theory-mathew-crawford-pdf-free',
 'Euclidean Geometry in Mathematical Olympiads by Evan Chen (z-lib.org)',
 'Proofs_That_Really_Count_The_Art_of_Combinatorial_',
 'Secrets in Inequalities (volume 1) Pham Kim Hung',
 'The art of problem solving. Volume 1, The basics ( PDFDrive )',
 '75427434-problem-books-in-mathematics-problem-solving-strategies',
 'dokumen.pub_competition-math-for-middle-school-later-printing-used-1441488871-9781441488879',
 'Functional Equations and How To Solve Them - Christopher G. Small',
 'paul-zeitz-

In [ ]:
import asyncio
from lib.question_generator import QuestionGenerator
from lib.question_generator.models import QuestionDifficulty
from lib.textbook_manager import TextbookManager
import os

# Initialize manager and generator
textbook_manager = TextbookManager()
generator = QuestionGenerator(model_name="deepseek-chat", output_dir="generated_questions/DS-MATH")

PAGES_PER_GROUP = 10      # Number of consecutive pages to combine for a single question generation call
MAX_CONCURRENT = 300      # Maximum concurrent tasks allowed
DELAY_BETWEEN_TASKS = 0.1 # Delay in seconds between scheduling tasks

async def process_pages(textbook_name, start_page, num_pages):
    """Process multiple consecutive pages and generate questions from their combined content."""
    combined_passage = None
    
    # Combine content from each page in the group
    for page_num in range(start_page, start_page + num_pages):
        current_passage = textbook_manager.get_page(textbook_name, page_number=page_num)
        if current_passage:
            if combined_passage is None:
                combined_passage = current_passage
            else:
                combined_passage.content += "\n\n" + current_passage.content
        else:
            print(f"Warning: Could not find page {page_num} in {textbook_name}")
    
    if combined_passage is None:
        print(f"Warning: Could not load any pages starting from {start_page}")
        return []
    
    print(f"Processing pages {start_page}-{start_page + num_pages - 1} of {textbook_name}...")
    try:
        questions = await generator.generate_questions(
            combined_passage.content,
            num_questions=6,
            difficulty=QuestionDifficulty.UNDERGRAD,
            verify=False,
            src=f"{textbook_name}_pages_{start_page}-{start_page + num_pages - 1}"
        )
        print(f"✓ Completed pages {start_page}-{start_page + num_pages - 1} of {textbook_name} - generated {len(questions)} questions")
        return questions
    except Exception as e:
        print(f"Error processing pages {start_page}-{start_page + num_pages - 1} of {textbook_name}: {str(e)}")
        return []

async def process_textbook(textbook_name):
    num_pages = textbook_manager.get_num_pages(textbook_name)
    print(f"\nProcessing textbook: {textbook_name}")
    print(f"Number of pages: {num_pages}")
    
    # Create a semaphore for controlling max concurrency per textbook
    semaphore = asyncio.Semaphore(MAX_CONCURRENT)
    
    # Wrap each task with the semaphore
    async def sem_task(start_page, num_pages):
        async with semaphore:
            return await process_pages(textbook_name, start_page, num_pages)
    
    # Create tasks for each group of pages (each group processes PAGES_PER_GROUP pages)
    tasks = []
    for group_start in range(0, num_pages, PAGES_PER_GROUP):
        actual_pages = min(PAGES_PER_GROUP, num_pages - group_start)
        tasks.append(sem_task(group_start, actual_pages))
        # Add a delay between scheduling each task
        await asyncio.sleep(DELAY_BETWEEN_TASKS)
    
    textbook_questions = []
    results = await asyncio.gather(*tasks)
    for group_questions in results:
        textbook_questions.extend(group_questions)
    
    print(f"\nFinished processing {textbook_name}. Total questions: {len(textbook_questions)}")
    return textbook_questions

async def main():
    # Get all textbook names from the "textbooks/txt" directory
    textbook_names = [f.replace('.txt', '') for f in os.listdir('textbooks/txt') if f.endswith('.txt')]
    print(f"Found {len(textbook_names)} textbooks: {textbook_names}")
    
    all_questions = []
    # Process each textbook concurrently
    tasks = [process_textbook(name) for name in textbook_names]
    results = await asyncio.gather(*tasks)
    for textbook_questions in results:
        all_questions.extend(textbook_questions)
    
    print(f"\nFinished processing all textbooks. Total questions generated: {len(all_questions)}")

# Run the main function in a notebook-friendly way:
await main()


Found 120 textbooks: ['Probability theory the logic of science (E. T. Jaynes, G. Larry Bretthorst) (z-lib.org)', 'Bruce Alberts, Alexander Johnson, Julian Lewis, Martin Raff, Keith Roberts, Peter Walter - Molecular Biology of the Cell, 5th edition-Garland Science (2007)', 'mth(40)', 'mth(17)', 'Dictionary of Algebra, Arithmetic, and Trigonometry', 'mth(56)', 'mth(21)', 'mth(60)', 'mth(37)', 'mth(2)', 'vector_calc', 'The Princeton Companion to Mathematics', 'Tom Apostol - Calculus vol.1 - One-variable Calculus, with an Introduction to Linear Algebra (1975)', 'mth(36)', 'mth(61)', 'mth(57)', 'Applied Combinatorics (6th Edition) by Alan Tucker 2012 PDF', 'mth(16)', 'mth(41)', 'szprecalc08262010', 'Formal Logic', 'Chemistry A Molecular Approach (5th Edition) by Nivaldo J. Tro (z-lib.org)', 'mth(27)', 'mth(66)', 'mth(31)', 'mth(4)', 'Discrete Mathematics with Applications 4th edition by Susanna Epp', 'A First Course in Probability (Sheldon M. Ross) (z-lib.org)', 'mth(46)', 'mth(8)', 'mth(11

In [12]:
for question in verified_questions:
    print(question.source)
    print(question.question)
    print(question.solution)
    print(question.hints)
    
    print("-" * 80)
print(len(verified_questions))

Millikan calculated the charge on oil droplets falling in an electric field. He found that it was always a whole-number multiple of -1.60*10-19 C, the charge of a single electron.
What was the charge of the oil droplets measured by Millikan, and how did he determine that it was a whole-number multiple?
Millikan measured the charge on oil droplets falling in an electric field and found that it was always a whole-number multiple of -1.60 x 10^-19 C, which is the charge of a single electron. He determined this by measuring the strength of the electric field required to halt the free fall of the negatively charged droplets and calculating the charge based on the mass of the droplets, which he derived from their radii and density.
['Start by reviewing the concept of electric charge and how it can be quantified. Consider what it means for a charge to be a whole-number multiple and how this relates to the fundamental charge of an electron.', 'Think about the experimental setup Millikan used w